## Display the xml element tree

In [ ]:
from pptlayout.extractors.run_extractors import run_extractors
from json import dumps
# Replace with your actual PPTX file path
pptx_path = "/data/tianyuhu/PPTLayout/data/pptx/ZK7FNUZ33GBBCG7CFVYS56TQCTD72CJR.pptx"

# Run extractors
info = run_extractors(pptx_path, "emu")
print(dumps(info, indent=4))

In [ ]:
from pptlayout.llm.call_ollama import generate_slide_layout_suggestions
from pptlayout.llm.prompts import build_slide_layout_suggestion_prompts
from tqdm import tqdm
import os
# Call OLLAMA
output_dir = os.path.join(os.curdir, "test_output")
suggestions_list = []
for slide_info in tqdm(info["slides"]):
    slide_id = slide_info["slide_id"]
    slide_output_dir = os.path.join(output_dir, f"{slide_id}")
    if os.path.exists(os.path.join(slide_output_dir, "suggestions.txt")):
        with open(os.path.join(slide_output_dir, "suggestions.txt"), "r") as f:
            suggestions = f.read()
        suggestions_list.append(suggestions)
        continue
    prompt = build_slide_layout_suggestion_prompts(
        json_input=slide_info, 
        slide_width=info["slide_width"],
        slide_height=info["slide_height"]
        )
    suggestions = generate_slide_layout_suggestions(
        model_name="llama3.1:70b",
        prompt=prompt,
        temperature=0.5,
        )
    suggestions_list.append(suggestions)
    # save
    os.makedirs(slide_output_dir, exist_ok=True)
    with open(os.path.join(slide_output_dir, "suggestions.txt"), "w") as f:
        f.write(suggestions)

In [ ]:
from pptlayout.llm.parser import extract_json
json_list = []
for suggestions in suggestions_list:
    json_data = extract_json(suggestions)
    json_list.append(json_data)

In [ ]:
from json import dumps

# print(dumps(json_list[12], indent=4))
for json in json_list:
    print(json)

In [ ]:
from pptlayout.visualizers.layout_visualizer import generate_comparison_grid

slides = info["slides"]
# Generate comparison grid
comparison_grid = generate_comparison_grid(
    slides,
    json_list,
    slide_width=info["slide_width"],
    slide_height=info["slide_height"]
    )

In [ ]:
from pptlayout.llm.call_ollama import generate_slide_layout_suggestions
test_image = "/data/tianyuhu/PPTLayout/notebooks/test_input/image.png"
import ollama
# Call OLLAMA
response = ollama.chat(
    model='llama3.2-vision:11b',
    messages=[{
        'role': 'user',
        'content': 'Please give me a detailed layout suggestion for this slide.',
        'images': [test_image]
    }]
)["message"]["content"]

print(response)


In [2]:
import aspose.slides as slides

pres = slides.Presentation("/data/tianyuhu/PPTLayout/data/pptx/ZK7FNUZ33GBBCG7CFVYS56TQCTD72CJR.pptx")
out_dir = "/data/tianyuhu/PPTLayout/notebooks/test_output/jpg"
for sld in pres.slides:
    with sld.get_image(1, 1) as bmp:
        bmp.save("{out_dir}/Slide_{num}.jpg".format(out_dir=out_dir, num=str(sld.slide_number)), slides.ImageFormat.JPEG)


RuntimeError: Proxy error(TypeInitializationException): The type initializer for 'Gdip' threw an exception. ---> DllNotFoundException: Unable to load shared library 'libgdiplus' or one of its dependencies. In order to help diagnose loading problems, consider setting the LD_DEBUG environment variable: liblibgdiplus: cannot open shared object file: No such file or directory